In [ ]:
# Pkg.add("JSON3")
# Pkg.add("DataFrames")
# Pkg.add("JSONTables")

In [ ]:
TargetPair = Tuple{Int, Int, Int} # x, y, speed

In [ ]:
import JSON3
using Pipe: @pipe
FILES_R = [
    "open_close_auto_radar_20240510_173207_88ms.jsonl",
    "open_close_auto_radar_20240510_173300_88ms.jsonl",
    "open_close_auto_radar_20240510_173518_88ms.jsonl",
    "move_vertical_short_20240510_112316_88ms.jsonl",
    "radar_20240510_113056_88ms.jsonl",
]
FILES = [
    "data/$(f)" for f in FILES_R
]
FILE_NAME = FILES[5]
global_targets::Vector{Array{TargetPair}} = []
# JSON3.readjsonlines
# https://stackoverflow.com/questions/67348301/julia-iterator-which-parses-each-line-in-file

open(FILE_NAME, "r") do f
    for m in Iterators.map(JSON3.read, eachline(f))
        targets = m["targets"]
        if targets == nothing || targets == []
            push!(global_targets, [])
            continue
        end
        function get_target(t)
            x = t["coord"][1]
            y = t["coord"][2]
            speed = t["speed"]
            return (x, y, speed)
        end
        ts = Iterators.map(get_target, targets)
        push!(global_targets, collect(ts))
    end
end

In [ ]:
NullableTargetPair = Union{Nothing,TargetPair}
function get_column(
    targets::AbstractArray{Array{TargetPair}},
    col::Int,
)::Vector{NullableTargetPair}
    return map(t -> length(t) < col ? nothing : t[col], targets)
end

fst_col = get_column(global_targets, 1)
snd_col = get_column(global_targets, 2)
thd_col = get_column(global_targets, 3)

non_nothing_length = xs -> filter(x -> x != nothing, xs) |> collect |> length

println("total_length=$(length(global_targets))")
println("fst non_empty_length=$(non_nothing_length(fst_col))")
println("snd non_empty_length=$(non_nothing_length(snd_col))")
println("thd non_empty_length=$(non_nothing_length(thd_col))")

In [ ]:
function segment_continous(xs::Vector{NullableTargetPair})::Vector{Vector{TargetPair}}
    segments = []
    current_segment = []
    for x in xs
        if x == nothing
            if length(current_segment) > 0
                push!(segments, current_segment)
                current_segment = []
            end
        else
            push!(current_segment, x)
        end
    end

    # end piece check
    if length(current_segment) > 0
        push!(segments, current_segment)
    end
    return segments
end

fst_segments = segment_continous(fst_col)
each_length = map(length, fst_segments)
println("each_length=$(each_length)")

In [ ]:
# let's plot the first segment, but only x and y
using Plots
get_coord = t -> (t[1], t[2])
get_speed = t -> t[3]

seg = fst_segments[1]
plot(seg |> xs -> map(get_coord, xs), seriestype = :scatter, label = "coord", aspect_ratio = 1)

In [ ]:
plot(seg |> xs -> map(get_speed, xs), seriestype = :line, label = "speed")